# End-to-End Tutorial for Metriq-Gym

In this tutorial, we will provide a comprehensive end-to-end example of how to use the **metriq-gym** package to run a quantum benchmark. This will include setting up the environment, defining a benchmark, and executing it on a quantum device.

In [1]:
from types import SimpleNamespace
from dotenv import load_dotenv
from metriq_gym.run import dispatch_job, poll_job, view_job
from metriq_gym.job_manager import JobManager

First, ensure you have a local `.env` file in the root directory of the project with your device credentials for your
account specified. The format of this file can be found in `.env.example`. Assuming you have the necessary credentials,
the next line will load them into the environment.

Please refer to the [user guide](https://metriq-gym.readthedocs.io/en/latest/user_guide.html) or README for more details on how to set up your environment.

In [2]:
# Load environment variables from .env file.
load_dotenv()

False

## Job handling

Metriq-Gym provides a simple API to dispatch jobs to quantum devices and to poll and view their status.

### Dispatch job

We can use the Python interface to dispatch a job to a quantum device or local simulator. For the sake of example, we will use the local simulator, but the `provider_name` and `device_name` can be changed to target a specific quantum device (assuming you have the necessary credentials in your `.env` file).

Metriq-Gym provides a suite of benchmarks that can be used to test the performance of quantum devices. In this example, we will use the `wit` (Wormhole-inspired teleportation) benchmark, which is a simple benchmark that tests the performance of a quantum device on a specific task. The specification for this benchmark can be found in the `metriq_gym/schemas/examples/wit.example.json` file.

More information about the `wit` (Wormhole-inspired teleportation) benchmark can be found in `benchmarks/wit.py` as well as in the paper on which it is based: [arXiv:2205.14081](https://arxiv.org/abs/2205.14081).

In [3]:
job_manager = JobManager()
benchmark_path = "../../metriq_gym/schemas/examples/wit.example.json"
provider_name = "local"
device_name = "aer_simulator"

dispatch_config = SimpleNamespace(
    config=benchmark_path,
    provider=provider_name,
    device=device_name,
)

dispatch_job(dispatch_config, job_manager)

Starting dispatch on local:aer_simulator...
Dispatching WIT...
✓ WIT dispatched with metriq-gym Job ID: 7cb5b2df-e62d-423f-ac22-4bf6739d2ea4


We can see that the `wit` (Wormhole-inspired teleportation) benchmark has been successfully dispatched to the local simulator. Dispatching a job creates a local `.jsonl` jobs data file that captures all of the metadata of the job we just launched. 

### View job

Now that we have dispatched the job, we can poll its status. The `JobManager` class provides a simple interface to poll the status of jobs. We can use the `get_jobs` method to get the list of jobs and their statuses. In this example, we will poll the status of the job we just dispatched.

In [4]:
jobs = job_manager.get_jobs()
job_id_to_poll = jobs[-1].id

job_config = SimpleNamespace(job_id=job_id_to_poll)

view_job(job_config, job_manager)

╒══════════════════╤════════════════════════════════════════════════════════════════╕
│ suite_id         │                                                                │
├──────────────────┼────────────────────────────────────────────────────────────────┤
│ id               │ 7cb5b2df-e62d-423f-ac22-4bf6739d2ea4                           │
├──────────────────┼────────────────────────────────────────────────────────────────┤
│ job_type         │ WIT                                                       │
├──────────────────┼────────────────────────────────────────────────────────────────┤
│ params           │ {'benchmark_name': 'WIT', 'shots': 8192, 'n_qubits_per_side': 3, 'message_size': 1, 'insert_message_method': 'swap', 'interaction_coupling_strength': 1.5707963267948966, 'x_rotation_transverse_angle': 0.7853981633974483, 'zz_rotation_angle': 0.7853981633974483, 'z_rotation_angles': [0.0283397, 0.00519953, 0.0316079], 'time_steps': 3, 'num_qubits': 7}' │
├──────────────────┼──────

### Poll job

Polling a job allows us to check its status and retrieve the results once it has completed. The `poll_job` function can be used to poll the job until it is completed.

In [5]:
poll_job(job_config, job_manager)

Polling job...
{'app_version': '0.3.1.post11.dev0',
 'device': 'aer_simulator',
 'job_type': 'WIT',
 'provider': 'local',
 'results': {'expectation_value': 0.9970703125},
 'suite_id': None,
 'timestamp': '2025-08-26T09:54:47.904520'}


In the case of the `wit` (Wormhole-inspired teleportation) benchmark, we can see that the job has been successfully dispatched and polled. The polled result contains information about when the result was obtained, any relevant results from the benchmark, etc. The ideal (or perfect) result expectation value for the `wit` (Wormhole-inspired teleportation) benchmark is `1.0` and we can see that the simulated result is very close to this value. 

Explaining the `wit` (Wormhole-inspired teleportation) benchmark in detail is beyond the scope of this tutorial, but it is a simple benchmark that tests the performance of a quantum device on a specific task. The specification for this benchmark can be found in the `metriq_gym/schemas/examples/wit.example.json` file.